In [1]:
import findspark
findspark.init()

In [3]:
from pyspark import SparkContext 
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.appName("Particionado") \
    .master("local[5]").getOrCreate()

In [6]:
df = spark.range(0,20)


In [7]:
df.rdd.getNumPartitions()

5

Una vez hecho lo anterior lo que veremos es el número de particiones que realizo spark sin que nostros se lo indiquemos.

In [10]:
rdd1 = spark.sparkContext.parallelize((0,20), 10)


In [11]:
rdd1.getNumPartitions()

10

In [12]:
!ls resources/files/

deporte.csv           deportistaError.csv   modelo_relacional.jpg
deportista.csv        evento.csv            paises.csv
deportista2.csv       juegos.csv            resultados.csv


In [13]:
rddDesdeArchivo = spark \
    .sparkContext \
    .textFile("resources/files/deporte.csv", 10) 

In [14]:
rddDesdeArchivo.getNumPartitions()

10

In [16]:
rddDesdeArchivo.saveAsTextFile("particiones/salidatext")

In [18]:
!ls particiones/salidatext/


_SUCCESS   part-00001 part-00003 part-00005 part-00007 part-00009
part-00000 part-00002 part-00004 part-00006 part-00008


In [19]:
!head particiones/salidatext/part-00000

deporte_id,deporte
1,Basketball
2,Judo
3,Football
4,Tug-Of-War
5,Speed Skating
6,Cross Country Skiing


In [20]:
rdd= spark.sparkContext.wholeTextFiles("particiones/salidatext/*")

In [21]:
rdd.take(4)

[('file:/Users/davidmorales/Documents/Platzi/Spark_2/particiones/salidatext/part-00000',
  'deporte_id,deporte\n1,Basketball\n2,Judo\n3,Football\n4,Tug-Of-War\n5,Speed Skating\n6,Cross Country Skiing\n'),
 ('file:/Users/davidmorales/Documents/Platzi/Spark_2/particiones/salidatext/part-00001',
  '7,Athletics\n8,Ice Hockey\n9,Swimming\n10,Badminton\n11,Sailing\n12,Biathlon\n13,Gymnastics\n14,Art Competitions\n'),
 ('file:/Users/davidmorales/Documents/Platzi/Spark_2/particiones/salidatext/part-00002',
  '15,Alpine Skiing\n16,Handball\n17,Weightlifting\n18,Wrestling\n19,Luge\n20,Water Polo\n'),
 ('file:/Users/davidmorales/Documents/Platzi/Spark_2/particiones/salidatext/part-00003',
  '21,Hockey\n22,Rowing\n23,Bobsleigh\n24,Fencing\n25,Equestrianism\n26,Shooting\n27,Boxing\n28,Taekwondo\n')]

In [22]:
lista = rdd.mapValues(lambda x : x.split()).collect()

In [24]:
lista = [l[0] for l in lista]
lista.sort()

In [25]:
lista

['file:/Users/davidmorales/Documents/Platzi/Spark_2/particiones/salidatext/part-00000',
 'file:/Users/davidmorales/Documents/Platzi/Spark_2/particiones/salidatext/part-00001',
 'file:/Users/davidmorales/Documents/Platzi/Spark_2/particiones/salidatext/part-00002',
 'file:/Users/davidmorales/Documents/Platzi/Spark_2/particiones/salidatext/part-00003',
 'file:/Users/davidmorales/Documents/Platzi/Spark_2/particiones/salidatext/part-00004',
 'file:/Users/davidmorales/Documents/Platzi/Spark_2/particiones/salidatext/part-00005',
 'file:/Users/davidmorales/Documents/Platzi/Spark_2/particiones/salidatext/part-00006',
 'file:/Users/davidmorales/Documents/Platzi/Spark_2/particiones/salidatext/part-00007',
 'file:/Users/davidmorales/Documents/Platzi/Spark_2/particiones/salidatext/part-00008',
 'file:/Users/davidmorales/Documents/Platzi/Spark_2/particiones/salidatext/part-00009']

In [26]:
rdd = spark \
    .sparkContext \
    .textFile(','.join(lista), 10).map(lambda l: l.split(","))

In [28]:
rdd.take(4)

[['deporte_id', 'deporte'],
 ['1', 'Basketball'],
 ['2', 'Judo'],
 ['3', 'Football']]